In [11]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.8 MB/s eta 0:00:00


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches

# Memuat dataset
data = pd.read_csv('/content/Electric_Production.csv')

# Mengonversi kolom tanggal menjadi datetime dan menjadikannya indeks
date_column = 'DATE'  # Nama kolom tanggal yang benar
value_column = 'IPG2211A2N'  # Nama kolom nilai yang sesuai

data[date_column] = pd.to_datetime(data[date_column])
data = data.set_index(date_column)

# Menghitung SMA, CMA, dan EMA
sma_window_size = 30  # Ganti dengan ukuran jendela yang diinginkan
ema_span = 30  # Ganti dengan periode span yang diinginkan

data['SMA'] = data[value_column].rolling(window=sma_window_size).mean()
data['CMA'] = data[value_column].expanding().mean()
data['EMA'] = data[value_column].ewm(span=ema_span, adjust=False).mean()

# Membuat dokumen Word
doc = Document()
doc.add_heading('Analisis Time Series', 0)

# Menambahkan Pendahuluan
doc.add_heading('Pendahuluan', level=1)
doc.add_paragraph(
    "Dataset yang digunakan adalah data produksi listrik. "
    "Analisis ini bertujuan untuk menghitung dan membandingkan Simple Moving Average (SMA), "
    "Cumulative Moving Average (CMA), dan Exponential Moving Average (EMA)."
)

# Menambahkan Metodologi
doc.add_heading('Metodologi', level=1)
doc.add_paragraph(
    "SMA dihitung dengan ukuran jendela 30 hari. "
    "CMA dihitung sebagai rata-rata kumulatif dari seluruh data. "
    "EMA dihitung dengan span 30 hari, yang memberi lebih banyak bobot pada nilai terbaru."
)

# Menambahkan Hasil
doc.add_heading('Hasil', level=1)

# Menambahkan Visualisasi
plt.figure(figsize=(12, 8))

# Plot data asli dan moving averages
plt.plot(data.index, data[value_column], label='Data Asli', color='blue')
plt.plot(data.index, data['SMA'], label=f'SMA {sma_window_size}-hari', color='red')
plt.plot(data.index, data['CMA'], label='CMA', color='green')
plt.plot(data.index, data['EMA'], label=f'EMA {ema_span}-hari', color='orange')

plt.title('Analisis Moving Average')
plt.xlabel('Tanggal')
plt.ylabel('Nilai')
plt.legend()
plt.grid(True)
plt.savefig('/content/time_series_analysis.png')
plt.close()

# Menambahkan gambar ke dokumen
doc.add_paragraph('Grafik di bawah ini menunjukkan perbandingan antara data asli dan berbagai moving averages:')
doc.add_picture('/content/time_series_analysis.png', width=Inches(6.0))

# Menambahkan Kesimpulan
doc.add_heading('Kesimpulan', level=1)
doc.add_paragraph(
    "Analisis menunjukkan bahwa SMA memberikan rata-rata bergerak sederhana selama periode tertentu, "
    "sementara CMA memberikan rata-rata kumulatif dari seluruh data. "
    "EMA memberikan bobot lebih pada data terbaru, sehingga lebih responsif terhadap perubahan data."
)

# Menyimpan dokumen
doc.save('/content/analisis_time_series.docx')
